In [ ]:
!pip install pandas requests

# REST API Examples

In this notebook we explore searching for metadata from the REST API. The REST API provides a method to programmatically extract a JSON representation of the meta data from the API.

First we load some python dependancies that we will use as part of this notebook and set the variable `API_URL` to the location of the REST API.

In [ ]:
import pandas as pd
import requests
API_URL = "https://mastapp.site/json"

Below we print the location of the endpoint we will query in this notebook:

In [ ]:
print(f"REST API Endpoint: {API_URL}")

## Querying Shots with the REST API

We're going to use the python `requests` library to query metadata from the database. All we need to do to get a result is to query the database with a HTTP GET at the appropriate endpoint. For example, to get information about different experimental shots we can query the `/json/shots/` endpoint.

In [ ]:
response = requests.get(f'{API_URL}/shots')
result = response.json()
print(f"Query returned status code: {response.status_code}")

The shots endpoint returns a JSON payload with a list of shots. Let's look at the first element from the payload:

In [ ]:
print(result['items'][0])

Each item in the list is a json object. This contains the meta-data items that corresponded to our query. In this case, each item contains information about a different MAST shot. Each item has lots of information about different shots, for example the shot ID, the campaign the shot was part of, the pre- and post-shot description by investigators.

For more information on the what's returned by the API you can look at the endpoint documentation:

https://mastapp.site/redoc


Of course, we can read all this JSON data directly into common python data analysis packages, for example, we can create a `pandas` dataframe directly from the endpoint data

In [ ]:
df = pd.read_json(f'{API_URL}/shots', lines=True)['items'][0]
df = pd.DataFrame(df)
df.head()



### Searching & Filtering Data

All REST API endpoints can take query parameters to filter the data returned. For example, we can return all shots for the `M9` campaign by using the approrpiate query string.

We can query for everything from the `M9` campaign by adding `?filters=campaign$eq:M9` to our query string.

In [ ]:
df = pd.read_json(f'{API_URL}/shots?filters=campaign$eq:M9', lines=True)['items'][0]
df = pd.DataFrame(df)
df.head()

### Pagination 
The REST API responses are _paginated_, meaning that only a subset of the full items are returned with each query. Pagination is used to limit the total number of requests made by each user to prevent any single user overloading the server with huge data requests.

Pagination information is included in the response and corresponds to [RFC 8288](https://datatracker.ietf.org/doc/html/rfc8288). The response contains cursors at the bottom of the response:

 - `current_page` - this is the cursor relating to the current page of results.
 - `next_page` - this is the cursor for the next page of results.
 - `previous_page` - this is the cursor for the previous page of results, will result in `null` if on the first page.

You can control the pagination using the following options as query arguments:
 - `cursor=xx` to set the cursor for the displayed set of results
 - `size=xx` to set the number of results returned by each page.

In [ ]:
response = requests.get(f'{API_URL}/shots?size=2&cursor=Pmk6MzAxMTE%3D')
result = response.json()
headers = response.headers

print("Current page cursor", result['current_page'])
print("Next page cursor", result['next_page'])
print("Previous page cursor", result['previous_page'])

df = pd.DataFrame(result['items'])
df